In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle, os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pretty_midi
import json

In [ ]:
plt.style.use('ggplot')

# track separation


## the track_separate.py can find melody, bass, chord, accompiment and drum tracks. 


The track separation model is a random forest trained each for melody, bass, chord and drum.

The accompaniment track is found based on the duration of the track, and it selects the longest duration in the left tracks.


the parameters includes:
-f input file
-i input folder (will search all the midi file in that folder)
The user need to specify one of the above two parameters 
-o output folder
-t required tracks to be found. It will always try to find melody, bass, chord, accompaniment and drum tracks, and -t specifies the mandatory tracks. e.g. "melody bass chord" means melody, bass and chord tracks are mandatory for the output. If it cannot find those tracks that file will be omitted. But if the accomaniment and drum are not found and melody, bass, chord are found that file will still be output. The default is "melody", which means only the melody is mandatory for the output.




### example of track_separate.py

read in the original file

In [ ]:
original_file = 'example/input/55269aebbc2e7784ca2d1d4472141889.mid'
original_pm = pretty_midi.PrettyMIDI(original_file)
print(f'the original file has {len(original_pm.instruments)} tracks')

In [ ]:
!python3 track_separate.py -h

In [ ]:
!python3 track_separate.py -f example/input/55269aebbc2e7784ca2d1d4472141889.mid -o example/output/ -t "melody bass chord"

The output file is in the example/output/ folder

In [ ]:
output_file = 'example/output/55269aebbc2e7784ca2d1d4472141889.mid'
output_pm = pretty_midi.PrettyMIDI(output_file)
print(f'the output file has {len(output_pm.instruments)} tracks')

the program_result.json has record of the program number of each track

In [ ]:
with open(os.path.join('example/output','program_result.json'), 'r') as fp:
    programs = json.load(fp)

In [ ]:
for key in programs.keys():
    for name,value in programs[key].items():
        print(f'{name} program number is {value}')
        

# tension calculation

The tension calculation is based on the spiral array theory (https://dspace.mit.edu/handle/1721.1/9139,
https://qmro.qmul.ac.uk/xmlui/bitstream/handle/123456789/11798/Herremans%20Tension%20ribbons%20Quantifying%202016%20Accepted.pdf?sequence=1) 


It maps the pitch to a 3-dim position in spiral array space, and keeps property in that space such as dist(perfect fifth) has shortest space in all the interval pairs.

In [ ]:
from tension_calculation import *

all the interval distance to C note in a scale

the vertical step is a parameter to set the note position in the 3d space
the original paper uses math.sqrt(2/15) which makes the interval distance of a major third equals a perfect fifth distance
Any value from math.sqrt(2/15) to math.sqrt(0.2) can work according to the original paper
Here 0.4 is used to make the perfect fifth has the shortest distance


## some properties of the distance in the spiral array space

In [ ]:
note_to_note_diff = note_to_note_pos([0,1,2,3,4,5,6,7,8,9,10,11],pitch_index_to_position(note_index_to_pitch_index[0]))

use ['C','D-','D','E-','E','F','F#','G','A-','A','B-','B'] to map the note to pitch names

In [ ]:
pitch_names = ['C','D-','D','E-','E','F','F#','G','A-','A','B-','B']

In [ ]:
for num, pitch_name in enumerate(pitch_names):
    print(f'the distance from {pitch_name} to C is {note_to_note_diff[num]}')

In [ ]:
note_to_key_diff = note_to_key_pos([0,1,2,3,4,5,6,7,8,9,10,11],major_key_position(0))

In [ ]:
for num, pitch_name in enumerate(pitch_names):
    print(f'the distance from {pitch_name} to key pos major C  is {note_to_key_diff[num]}')

In [ ]:
note_to_key_diff = note_to_key_pos([0,1,2,3,4,5,6,7,8,9,10,11],minor_key_position(3))

In [ ]:
for num, pitch_name in enumerate(pitch_names):
    print(f'the distance from {pitch_name} to key pos minor a  is {note_to_key_diff[num]}')

In [ ]:
chord_to_key_diff = chord_to_key_pos([0,1,2,3,4,5,6,7,8,9,10,11],major_key_position(0))

In [ ]:
chord_names = ['CM','D-M','DM','E-M','EM','FM','F#M','G-M','A-M','AM','B-M','BM',
              'Cm','D-m','Dm','E-m','Em','Fm','F#m','G-m','A-m','Am','B-m','Bm']

In [ ]:
for num, pitch_name in enumerate(chord_names):
    print(f'the distance from chord pos {chord_names[num]} to  major C key pos is {chord_to_key_diff[num]}')

In [ ]:
key_to_key_diff = key_to_key_pos([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], major_key_position(0))

In [ ]:
for num, pitch_name in enumerate(chord_names):
    print(f'the distance from key pos {chord_names[num]} to  major C key pos is {key_to_key_diff[num]}')

## calculate the tension by tension_calculation.py

In [ ]:
def draw_tension(time,values):
    fig = plt.figure(figsize=(20, 10))
    plt.rcParams['xtick.labelsize'] = 14
    plt.plot(time,values,marker='o')
    plt.tight_layout()
    plt.show()
   
  
    

three tension measures are calculated including tensile strain, cloud diameter and centroid difference (https://qmro.qmul.ac.uk/xmlui/bitstream/handle/123456789/11798/Herremans%20Tension%20ribbons%20Quantifying%202016%20Accepted.pdf?sequence=1)


To calculate those measures, the key of the song needs to be detected first. It finds the key by mappping the notes pos in all the keys and find the shortest distance in a key

It also tries to find one key change of the song which is common in pop music, but the classical music has more key change

In [ ]:
!python3 tension_calculation.py -h

this example, it will try to detect key change, use window size -1 (a downbeat i.e. bar window)

In [ ]:
!python3 tension_calculation.py -f example/output/55269aebbc2e7784ca2d1d4472141889.mid -o example/output -k True -w 1

this song should be in a minor, now set the key to a minor by -n "a minor"

In [ ]:
!python3 tension_calculation.py -f example/output/55269aebbc2e7784ca2d1d4472141889.mid -o example/output -k True -w 1 -n "a minor"

## cloud diameter result for bar/time x axis

In [ ]:
diameter = pickle.load(open('example/output/55269aebbc2e7784ca2d1d4472141889.diameter','rb'))
times = pickle.load(open('example/output/55269aebbc2e7784ca2d1d4472141889.time','rb'))

In [ ]:
print(f'the file has {len(diameter)} bar')

In [ ]:
print(f'the max diameter is located at bar {np.argmax(diameter) + 1} ')

x label is bar

In [ ]:
draw_tension(np.arange(diameter.shape[0]) + 1,diameter)

xlabel is time (s)

In [ ]:
draw_tension(times[:len(diameter)],diameter)

## tensile strain output for bar/time x axis

In [ ]:
tensile = pickle.load(open('example/output/55269aebbc2e7784ca2d1d4472141889.tensile','rb'))


In [ ]:
print(f'the max tensile strain is located at bar {np.argmax(tensile) + 1} ')

x label is bar

In [ ]:
draw_tension(np.arange(tensile.shape[0]) + 1,tensile)

xlabel is time (s)

In [ ]:
draw_tension(times[:len(tensile)],tensile)

## centroid difference output for bar/time x axis

In [ ]:
centroid_diff = pickle.load(open('example/output/55269aebbc2e7784ca2d1d4472141889.centroid_diff','rb'))


In [ ]:
print(f'the max centroid difference is located at bar {np.argmax(centroid_diff) + 1} ')

x label is bar

In [ ]:
draw_tension(np.arange(centroid_diff.shape[0])+1,centroid_diff)

xlabel is time (s)

In [ ]:
draw_tension(times[:len(centroid_diff)],centroid_diff)

In [ ]:
with open(os.path.join('example/output','files_result.json'), 'r') as fp:
    keys = json.load(fp)

In [ ]:
for key in keys.keys():
    print(f'song name is {key}')
    print(f'song key is {keys[key][0]}')
    print(f'song key change time {keys[key][1]}')
    print(f'song key change bar {keys[key][2]}')
    print(f'song key change name {keys[key][3]}')
    
    
        

-1 change time(or bar) means key no change